#   Airbnb Data Analysis Report


###   Prepare and Download DataSet


In [6]:
## initial directorys
!mkdir data/
!mkdir data/boston
!mkdir data/seattle

## download data from oss
!wget --no-check-certificate -c https://oss.hacking-linux.com/udacity/airbnb/seattle/calendar.csv -O data/seattle/calendar.csv
!wget --no-check-certificate -c https://oss.hacking-linux.com/udacity/airbnb/seattle/listings.csv -O data/seattle/listings.csv
!wget --no-check-certificate -c https://oss.hacking-linux.com/udacity/airbnb/seattle/reviews.csv -O data/seattle/reviews.csv
!wget --no-check-certificate -c https://oss.hacking-linux.com/udacity/airbnb/boston/calendar.csv -O data/boston/calendar.csv
!wget --no-check-certificate -c https://oss.hacking-linux.com/udacity/airbnb/boston/listings.csv -O data/boston/listings.csv
!wget --no-check-certificate -c https://oss.hacking-linux.com/udacity/airbnb/boston/reviews.csv -O data/boston/reviews.csv


--2018-09-05 12:12:39--  https://oss.hacking-linux.com/udacity/airbnb/seattle/calendar.csv
Resolving oss.hacking-linux.com (oss.hacking-linux.com)... 47.75.18.176, 47.75.18.190
Connecting to oss.hacking-linux.com (oss.hacking-linux.com)|47.75.18.176|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 36650955 (35M) [text/csv]
Saving to: 'data/seattle/calendar.csv'

data/seattle/calend 100%[===================>]  34.95M  11.1MB/s    in 3.2s    

2018-09-05 12:12:43 (11.1 MB/s) - 'data/seattle/calendar.csv' saved [36650955/36650955]

--2018-09-05 12:12:43--  https://oss.hacking-linux.com/udacity/airbnb/seattle/listings.csv
Resolving oss.hacking-linux.com (oss.hacking-linux.com)... 47.75.18.176, 47.75.18.190
Connecting to oss.hacking-linux.com (oss.hacking-linux.com)|47.75.18.176|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 16871061 (16M)

###   import pkgs

In [7]:
## import pkgs
import pandas as pd
import matplotlib.pyplot as plt

## magic funcs
%matplotlib inline

###   Declare Global Values


In [9]:
## define base dir
DATA_DIR = './data'
SEATTLE_DIR = '{}/seattle'.format(DATA_DIR)
BOSTON_DIR = '{}/boston'.format(DATA_DIR)

## define data file point
FP_SEATTLE_CLENDAR = '{}/calendar.csv'.format(SEATTLE_DIR)
FP_SEATTLE_LISTINGS = '{}/listings.csv'.format(SEATTLE_DIR)
FP_SEATTLE_REVIEWS = '{}/reviews.csv'.format(SEATTLE_DIR)

FP_BOSTON_CLENDAR = '{}/calendar.csv'.format(BOSTON_DIR)
FP_BOSTON_LISTINGS = '{}/listings.csv'.format(BOSTON_DIR)
FP_BOSTON_REVIEWS = '{}/reviews.csv'.format(BOSTON_DIR)


###   Load Data


In [32]:
## load data from csv
df_seattle_clendar_raw = pd.read_csv(FP_SEATTLE_CLENDAR)
df_seattle_listings_raw = pd.read_csv(FP_SEATTLE_LISTINGS)
df_seattle_reviews_raw = pd.read_csv(FP_SEATTLE_REVIEWS)

df_boston_clendar_raw = pd.read_csv(FP_BOSTON_CLENDAR)
df_boston_listings_raw = pd.read_csv(FP_BOSTON_LISTINGS)
df_boston_reviews_raw = pd.read_csv(FP_BOSTON_REVIEWS)


###   Explore Data


####   df_seattle_clendar_raw:


-   The table of df_seattle_clendar_raw's meaning:

|Feature|Meaning|Relation|
|:-:|:-:|:-:|
|listing_id|清单id||
|id|id||
|date|日期||
|reviewer_id|审核人id||
|reviewer_name|审核人名||
|comments|备注||


In [23]:
df_seattle_clendar_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84849 entries, 0 to 84848
Data columns (total 6 columns):
listing_id       84849 non-null int64
id               84849 non-null int64
date             84849 non-null object
reviewer_id      84849 non-null int64
reviewer_name    84849 non-null object
comments         84831 non-null object
dtypes: int64(3), object(3)
memory usage: 3.9+ MB


In [22]:
df_seattle_clendar_raw.head()


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...


####   df_seattle_listings_raw:


-   The table of df_seattle_listings_raw's meaning:

|Feature|Meaning|Relation|
|:-:|:-:|:-:|
|id|id||
|listing_url|清单url||
|scrape_id|||
|last_scraped|||
|name|清单名||
|summary|摘要||
|space|空间||
|description|描述||
|experiences_offered|体验||
|neighborhood_overview|邻居||
|notes|记录||
|transit|中转||
|thumbnail_url|简介url||
|medium_url|视频url||
|picture_url|图片url||
|xl_picture_url|x1图片url||
|host_id|租客id||
|host_url|租客url||
|host_name|租客姓名||
|host_since|租客||
|host_location|租客地址||
|host_about|租客信息||
|host_response_time|租客响应时间||
|host_response_rate|租客响应率||
|host_acceptance_rate|租客验收率||
|host_is_superhost|租客是会员||
|host_thumbnail_url|租客简介url||
|host_picture_url|租客图片url||
|host_neighbourhood|租客邻居||
|host_listings_count|租客清单数||
|host_total_listings_count|租客清单总数||
|host_verifications|租客验证信息||
|host_has_profile_pic|租客是否有头像||
|host_identity_verified|租客账户是否验证||
|street|街道||
|neighbourhood|邻居||
|neighbourhood_cleansed|||
|neighbourhood_group_cleansed|||
|city|城市||
|state|街道||
|zipcode|邮编||
|market|市场||
|smart_location|智能定位||
|country_code|国家代码||
|country|国家||
|latitude|纬度||
|longitude|经度||
|is_location_exact|位置是否准确||
|property_type|房屋种类||
|room_type|房型||
|accommodates|容纳空间||
|bathrooms|浴室||
|bedrooms|卧室||
|beds|床||
|bed_type|床类型||
|amenities|设施||
|square_feet|平方||
|price|价格||
|weekly_price|价格(周)||
|monthly_price|价格(月)||
|security_deposit|保证金||
|cleaning_fee|清洁费||
|guests_included|客人数||
|extra_people|额外的人||
|minimum_nights|最少的夜晚||
|maximum_nights|最多的夜晚||
|calendar_updated|日历已更新||
|has_availability|有空房||
|availability_30|可租(30天)||
|availability_60|可租(60天)||
|availability_90|可租(90天)||
|availability_365|可租(365天)||
|calendar_last_scraped|||
|number_of_reviews|评论数||
|first_review|第一条评论||
|last_review|最后一条评论||
|review_scores_rating|评论分数||
|review_scores_accuracy|评论分数(准确度)||
|review_scores_cleanliness|评论分数(清洁度)||
|review_scores_checkin|评论分数(入住)||
|review_scores_communication|评论分数(交流)||
|review_scores_location|评论分数(位置)||
|review_scores_value|评论分数(价值)||
|requires_license|需要许可证||
|license|许可证||
|jurisdiction_names|辖区名称||
|instant_bookable|即时预订||
|cancellation_policy|取消政策||
|require_guest_profile_picture|需要租客头像||
|require_guest_phone_verification|需要租客手机通过验证||
|calculated_host_listings_count|计算租客清单数量||
|reviews_per_month|每个月的评论||


In [36]:
df_seattle_listings_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 92 columns):
id                                  3818 non-null int64
listing_url                         3818 non-null object
scrape_id                           3818 non-null int64
last_scraped                        3818 non-null object
name                                3818 non-null object
summary                             3641 non-null object
space                               3249 non-null object
description                         3818 non-null object
experiences_offered                 3818 non-null object
neighborhood_overview               2786 non-null object
notes                               2212 non-null object
transit                             2884 non-null object
thumbnail_url                       3498 non-null object
medium_url                          3498 non-null object
picture_url                         3818 non-null object
xl_picture_url                      3498

In [63]:
df_seattle_listings_raw.head()


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,...,10.0,f,NaN,WASHINGTON,f,strict,f,f,2,1.15
3,7421966,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,none,NaN,...,NaN,f,NaN,WASHINGTON,f,flexible,f,f,1,NaN
4,278830,https://www.airbnb.com/rooms/278830,20160104002432,2016-01-04,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,...,9.0,f,NaN,WASHINGTON,f,strict,f,f,1,0.89


####   df_seattle_reviews_raw:


-   The table of df_seattle_reviews_raw's meaning:

|Feature|Meaning|Relation|
|:-:|:-:|:-:|
|listing_id|清单列表||
|id|id||
|date|日期||
|reviewer_id|评论id||
|reviewer_name|评论名||
|comments|备注||


In [38]:
df_seattle_reviews_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84849 entries, 0 to 84848
Data columns (total 6 columns):
listing_id       84849 non-null int64
id               84849 non-null int64
date             84849 non-null object
reviewer_id      84849 non-null int64
reviewer_name    84849 non-null object
comments         84831 non-null object
dtypes: int64(3), object(3)
memory usage: 3.9+ MB


In [39]:
df_seattle_reviews_raw.head()


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...


####   df_boston_clendar_raw:


-   The table of df_boston_clendar_raw's meaning:

|Feature|Meaning|Relation|
|:-:|:-:|:-:|
|listing_id|清单id||
|date|日期||
|available|有效性||
|price|价格||


In [41]:
df_boston_clendar_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308890 entries, 0 to 1308889
Data columns (total 4 columns):
listing_id    1308890 non-null int64
date          1308890 non-null object
available     1308890 non-null object
price         643037 non-null object
dtypes: int64(1), object(3)
memory usage: 39.9+ MB


In [42]:
df_boston_clendar_raw.head()


,listing_id,date,available,price
0,12147973,2017-09-05,f,NaN
1,12147973,2017-09-04,f,NaN
2,12147973,2017-09-03,f,NaN
3,12147973,2017-09-02,f,NaN
4,12147973,2017-09-01,f,NaN


####   df_boston_listings_raw:


-   The table of df_boston_listings_raw's meaning:

|Feature|Meaning|Relation|
|:-:|:-:|:-:|
|id|id||
|listing_url|清单url||
|scrape_id|||
|last_scraped|||
|name|清单名||
|summary|摘要||
|space|空间||
|description|描述||
|experiences_offered|体验||
|neighborhood_overview|邻居||
|notes|记录||
|transit|中转	||
|access|访问||
|interaction|影响||
|house_rules|房屋规则||
|thumbnail_url|简介url||
|medium_url|媒体url||
|picture_url|图片url||
|xl_picture_url|x1图片url||
|host_id|租客id||
|host_url|租客url||
|host_name|租客姓名||
|host_since|租客预定时间||
|host_location|租客地址||
|host_about|租客信息||
|host_response_time|租客响应时间||
|host_response_rate|租客响应率||
|host_acceptance_rate|租客验收率||
|host_is_superhost|租客是会员||
|host_thumbnail_url|租客简介url||
|host_picture_url|租客图片url||
|host_neighbourhood|租客邻居||
|host_listings_count|租客清单数||
|host_total_listings_count|租客清单总数||
|host_verifications|租客验证信息||
|host_has_profile_pic|租客是否有头像||
|host_identity_verified|租客账户是否验证||
|street|街道||
|neighbourhood|邻居||
|neighbourhood_cleansed|||
|neighbourhood_group_cleansed|||
|city|城市||
|state|街道||
|zipcode|邮编||
|market|市场||
|smart_location|智能定位||
|country_code|国家代码||
|country|国家||
|latitude|纬度||
|longitude|经度||
|is_location_exact|位置是否准确||
|property_type|房屋种类||
|room_type|房型||
|accommodates|容纳空间||
|bathrooms|浴室||
|bedrooms|卧室||
|beds|床||
|bed_type|床类型||
|amenities|设施||
|square_feet|平方||
|price|价格||
|weekly_price|价格(周)||
|monthly_price|价格(月)||
|security_deposit|保证金||
|cleaning_fee|清洁费||
|guests_included|客人数||
|extra_people|额外的人||
|minimum_nights|最少的夜晚||
|maximum_nights|最多的夜晚||
|calendar_updated|日历已更新||
|has_availability|有空房||
|availability_30|可租(30天)||
|availability_60|可租(60天)||
|availability_90|可租(90天)||
|availability_365|可租(365天)||
|calendar_last_scraped|||
|number_of_reviews|评论数||
|first_review|第一条评论||
|last_review|最后一条评论||
|review_scores_rating|评论分数||
|review_scores_accuracy|评论分数(准确度)||
|review_scores_cleanliness|评论分数(清洁度)||
|review_scores_checkin|评论分数(入住)||
|review_scores_communication|评论分数(交流)||
|review_scores_location|评论分数(位置)||
|review_scores_value|评论分数(价值)||
|requires_license|需要许可证||
|license|许可证||
|jurisdiction_names|辖区名称||
|instant_bookable|即时预订||
|cancellation_policy|取消政策||
|require_guest_profile_picture|需要租客头像||
|require_guest_phone_verification|需要租客手机通过验证||
|calculated_host_listings_count|计算租客清单数量||
|reviews_per_month|每个月的评论||


In [43]:
df_boston_listings_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3585 entries, 0 to 3584
Data columns (total 95 columns):
id                                  3585 non-null int64
listing_url                         3585 non-null object
scrape_id                           3585 non-null int64
last_scraped                        3585 non-null object
name                                3585 non-null object
summary                             3442 non-null object
space                               2528 non-null object
description                         3585 non-null object
experiences_offered                 3585 non-null object
neighborhood_overview               2170 non-null object
notes                               1610 non-null object
transit                             2295 non-null object
access                              2096 non-null object
interaction                         2031 non-null object
house_rules                         2393 non-null object
thumbnail_url                       2986

In [44]:
df_boston_listings_raw.head()


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,9.0,f,NaN,NaN,t,moderate,f,f,1,1.30
2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,The LOCATION: Roslindale is a safe and diverse...,...,10.0,f,NaN,NaN,f,moderate,t,f,1,0.47
3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,Roslindale is a lovely little neighborhood loc...,...,10.0,f,NaN,NaN,f,moderate,f,f,1,1.00
4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,"I love the proximity to downtown, the neighbor...",...,10.0,f,NaN,NaN,f,flexible,f,f,1,2.25


####   df_boston_reviews_raw:


-   The table of df_boston_eviews_raw's meaning:

|Feature|Meaning|Relation|
|:-:|:-:|:-:|
|listing_id|清单id||
|id|id||
|date|日期||
|reviewer_id|评论id||
|reviewer_name|评论名||
|comments|备注||


In [45]:
df_boston_reviews_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68275 entries, 0 to 68274
Data columns (total 6 columns):
listing_id       68275 non-null int64
id               68275 non-null int64
date             68275 non-null object
reviewer_id      68275 non-null int64
reviewer_name    68275 non-null object
comments         68222 non-null object
dtypes: int64(3), object(3)
memory usage: 3.1+ MB


In [46]:
df_boston_reviews_raw.head()


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,1178162,4724140,2013-05-21,4298113,Olivier,My stay at islam's place was really cool! Good...
1,1178162,4869189,2013-05-29,6452964,Charlotte,Great location for both airport and city - gre...
2,1178162,5003196,2013-06-06,6449554,Sebastian,We really enjoyed our stay at Islams house. Fr...
3,1178162,5150351,2013-06-15,2215611,Marine,The room was nice and clean and so were the co...
4,1178162,5171140,2013-06-16,6848427,Andrew,Great location. Just 5 mins walk from the Airp...


In [49]:
print('|Feature|Meaning|Relation|\n|:-:|:-:|:-:|')
for x in list(df_boston_reviews_raw.columns):
    print('|{}|||'.format(x))
    

|Feature|Meaning|Relation|
|:-:|:-:|:-:|
|listing_id|||
|id|||
|date|||
|reviewer_id|||
|reviewer_name|||
|comments|||
